# Topic

    Emotions increase our chances of survival by providing us with the ability to deal with sudden events in our surroundings.

    In a positve state, optimistic feelings dominate and cognitive functions (i.e. problem-solving, abilities.


## Interested Aspects / Analysis

### Questions 
1. Make predictions of timing:
  * Time that argument may happen 
  * Best time to approach each other
  * Time of possible impulsive decisions 
  * Time of tiredness (time for a break) 


2. Make predictions of the events: 
 * what event you are doing 


3. Notify the emotions: 
  * Showing the emotions right now (help them to link the event and the interactions with the emotions)


4. Give the notification for the possible outcome: 
  * (early warning) See the signals change, and give the instructions of what they should be doing in response to the situation (provide tailored advice for them to reach their set point at the moment)
  * (promote the positive experience) let them know they are relaxed and motivated, they should capture the moment < in a way I think the experiencing self can really help shape your experience and in turn shape the remembering self if we can give the instructions and the interventions at the moment)
  * Gives the warning if the signal shows the potential health issue <like if you are in danger and your physiological signal change dramatically they can contact your emergency contact directly)

### Analysis

1. Clustering with t-SNE and PCA on these features 
2. Seq-2-Seq for data every minute

# Data Preprocessing

In [1]:
import numpy as np 
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing import sequence
import datetime
sns.set(color_codes = True)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Using TensorFlow backend.


In [3]:
# Sampling frequency 64Hz ~= 0.015625 s
bvp = pd.read_csv('data/BVP.csv', header= None)
bvp.columns  = ['bvp']
print(len(bvp))
bvp.head()

2402644


,bvp
0,1.522203e+09
1,6.400000e+01
2,-0.000000e+00
3,-0.000000e+00
4,-0.000000e+00


In [8]:
print(bvp.iloc[0])

bvp    1.522203e+09
Name: 0, dtype: float64


In [10]:
init_time = datetime.datetime.utcfromtimestamp(bvp.iloc[0]).strftime('%Y-%m-%d %H:%M:%S')
init_time

'2018-03-28 02:05:40'

In [14]:
# Sampling frequency 4Hz ~= 0.25s
eda = pd.read_csv('data/EDA.csv', header= None)
eda.columns  = ['eda']
eda = eda.rolling(4).sum()
eda = eda.dropna().reset_index(drop = True)

print(len(eda))
print(eda.columns)
eda.head(10)

150161
Index(['eda'], dtype='object')


,eda
0,1.522203e+09
1,4.284421e+00
2,4.586610e-01
3,6.329010e-01
4,6.931160e-01
5,7.008030e-01
6,7.033650e-01
7,7.046460e-01
8,7.059270e-01
9,7.072090e-01


In [16]:
eda = eda.iloc[1:]
eda.head(5)

,eda
2,0.458661
3,0.632901
4,0.693116
5,0.700803
6,0.703365


In [ ]:
#create index 
index = []
for i in range(len(eda)):
    index.append(init_time + i*)

In [4]:
eda = eda.iloc[::4,:]
eda = eda.reset_index(drop = True)
eda = eda.dropna()
print(len(eda))
eda.head()

37541


,eda
0,1.522203e+09
1,6.931160e-01
2,7.059270e-01
3,7.110540e-01
4,7.072090e-01


In [5]:
# Sampling frequency 1Hz ~= 1s
hr = pd.read_csv('HR.csv', header= None)
hr.columns  = ['hr']

print(len(hr))
hr.head()

37534


,hr
0,1.522203e+09
1,1.000000e+00
2,5.500000e+01
3,6.400000e+01
4,6.000000e+01


In [6]:
# Sampling frequency 4Hz ~= 0.25s
temp = pd.read_csv('TEMP.csv', header= None)
temp.columns  = ['temp']
temp = temp.rolling(4).sum()

temp = temp.dropna().reset_index(drop = True)
temp.columns = ['temp']
print(len(temp))
print(temp.columns)
temp.head()

150159
Index(['temp'], dtype='object')


,temp
0,1.522203e+09
1,1.088500e+02
2,1.398000e+02
3,1.398000e+02
4,1.398000e+02


In [7]:
temp = temp.iloc[::4,:]
temp = temp.dropna().reset_index(drop = True)
print(len(temp))
temp.head()

37540


,temp
0,1.522203e+09
1,1.398000e+02
2,1.397600e+02
3,1.396400e+02
4,1.396000e+02


In [8]:
drop_eda = len(eda) - len(hr)
drop_temp = len(temp)  - len(hr)
eda = eda[:-drop_eda]
temp = temp[:-drop_temp]

In [9]:
print(len(eda))
print(len(hr))
print(len(temp))

37534
37534
37534


In [10]:
df = pd.concat([eda, hr, temp], axis = 1)
df.head()

,eda,hr,temp
0,1.522203e+09,1.522203e+09,1.522203e+09
1,6.931160e-01,1.000000e+00,1.398000e+02
2,7.059270e-01,5.500000e+01,1.397600e+02
3,7.110540e-01,6.400000e+01,1.396400e+02
4,7.072090e-01,6.000000e+01,1.396000e+02


## TensorFlow Dataset